### PISO FUERTE

In [1]:
import pandas as pd
import numpy as np
#import plotly.graph_objects as go
import yfinance as yf
import talib as ta
#import matplotlib.pyplot as plt
#import math
from datetime import date, timedelta
from scipy import stats
from scipy.signal import argrelextrema
from bokeh.plotting import figure, show, column
from sklearn.linear_model import LinearRegression
import os
from backtesting import Backtest, Strategy

/usr/share/miniconda/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
df_d = pd.read_csv('data/dataxd.txt', sep="\t")
df_h=pd.read_csv('data/dataxh.txt',sep='\t')

tickers = [
    'SPY',
    'META',
    'AAPL',
    'AMZN',
    'NFLX',
    'MRNA',
    'TSLA',
    'TNA',
    'GLD',
    'SLV',
    'USO',
    'BAC',
    'CVX',
    'XOM',
    'QQQ',
    'MSFT',
    'NVDA',
    'WMT',
    'BA',
    'DIS',
    'CAT',
    'IBM',
    'WFC',
    'PLTR',
    'AMD',
    'AVGO',
    'HOOD',
    'CRWV',
    'MSTR',
    'UNH',
    'GOOG',
    'APP',
    'UBER'
]

In [3]:
df_d=df_d.rename(columns={'datetime': 'fecha'})
df_d['fecha'] = pd.to_datetime(df_d['fecha'])
df_dia = pd.DataFrame()
for ticker in tickers:
    company = df_d.query("companyName==@ticker").copy()
    company.sort_values(by=['fecha'])
    company['SMA20'] = company['Close'].rolling(20).mean()
    company.dropna(inplace=False)
    company['SMA40'] = company['Close'].rolling(40).mean()
    company.dropna(inplace=False)
    company['SMA100'] = company['Close'].rolling(100).mean()
    company.dropna(inplace=False)
    company['SMA200'] = company['Close'].rolling(200).mean()
    company.dropna(inplace=False)
    df_dia = pd.concat([df_dia, company],ignore_index=True)

In [4]:
df_h['datetime'] = pd.to_datetime(df_h['datetime'])
df_h["fecha"] = pd.to_datetime(df_h["datetime"].dt.date)
df_hora = pd.DataFrame()
for ticker in tickers:
    company = df_h.query("companyName==@ticker").copy()
    company.sort_values(by=['datetime'])
    company['SMA20'] = company['Close'].rolling(20).mean()
    company.dropna(inplace=False)
    company['SMA40'] = company['Close'].rolling(40).mean()
    company.dropna(inplace=False)
    company['SMA100'] = company['Close'].rolling(100).mean()
    company.dropna(inplace=False)
    company['SMA200'] = company['Close'].rolling(200).mean()
    company.dropna(inplace=False)
    df_hora = pd.concat([df_hora, company],ignore_index=True)

#### Detect Pivots/Fractals

In [5]:
ord=17
max_idx1 = argrelextrema(df_dia['Close'].values, np.greater, order=ord)[0]
min_idx1 = argrelextrema(df_dia['Close'].values, np.less, order=ord)[0]
# Aplicar el cálculo solo a los índices en la lista
df_dia.loc[max_idx1, 'pivotHigh'] = df_dia['High']+1e-3
df_dia.loc[min_idx1, 'pivotLow'] = df_dia['Low']-(1e-3)
df_dia.loc[max_idx1, 'isPivot'] = 1
df_dia.loc[min_idx1, 'isPivot'] = 2

#HORA
max_idx2 = argrelextrema(df_hora['Close'].values, np.greater, order=ord)[0]
min_idx2 = argrelextrema(df_hora['Close'].values, np.less, order=ord)[0]
# Aplicar el cálculo solo a los índices en la lista
df_hora.loc[max_idx2, 'pivotHigh'] = df_hora['High']+1e-3
df_hora.loc[min_idx2, 'pivotLow'] = df_hora['Low']-(1e-3)
df_hora.loc[max_idx2, 'isPivot'] = 1
df_hora.loc[min_idx2, 'isPivot'] = 2

In [6]:
#CODIGO DE REVISION DIA
dfpl = df_dia.iloc[0:150]

p = figure(width=2500, height=500,
        title="DIA",
        background_fill_color="#efefef",
        tooltips=[("Index", "@index"), ("Open", "@Open"), ("High","@High"), ("Low","@Low"), ("Close","@Close")]
        )
p.xaxis.major_label_orientation = 0.8 # radians
p.x_range.range_padding = 0.05
p.xaxis.axis_line_width = 2

p.segment("index", "High", "index","Low",  color="black", line_width=1, source=dfpl)

inc = dfpl.query("Close>Open")
dec = dfpl.query("Open>Close")

p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="red",
    line_color="red",    
    source=dec   
)

p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="green",
    line_color="green", 
    source=inc   
)

p.scatter(x="index", y="pivotHigh", marker="circle", size=10,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="Cambio Tendencia Bajista", source=dfpl[(dfpl.isPivot==1)])

p.scatter(x="index", y="pivotLow", marker="circle", size=10,
           line_color="navy", fill_color="green", alpha=0.5, legend_label="Cambio Tendencia Alcista", source=dfpl[(dfpl.isPivot==2)])

show(p)

In [7]:
#CODIGO DE REVISION HORA
dfpl = df_hora.iloc[0:150]

p = figure(width=2500, height=500,
        title="HORA",
        background_fill_color="#efefef",
        tooltips=[("Index", "@index"), ("Open", "@Open"), ("High","@High"), ("Low","@Low"), ("Close","@Close")]
        )
p.xaxis.major_label_orientation = 0.8 # radians
p.x_range.range_padding = 0.05
p.xaxis.axis_line_width = 2

p.segment("index", "High", "index","Low",  color="black", line_width=1, source=dfpl)

inc = dfpl.query("Close>Open")
dec = dfpl.query("Open>Close")

p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="red",
    line_color="red",    
    source=dec   
)


p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="green",
    line_color="green", 
    source=inc   
)

p.scatter(x="index", y="pivotHigh", marker="circle", size=10,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="Cambio Tendencia Bajista", source=dfpl[(dfpl.isPivot==1)])

p.scatter(x="index", y="pivotLow", marker="circle", size=10,
           line_color="navy", fill_color="green", alpha=0.5, legend_label="Cambio Tendencia Alcista", source=dfpl[(dfpl.isPivot==2)])

show(p)

In [8]:
## cerca a PM100 o PM200 en diario o el PM20 bajo el PM40
def zonabarata(df_dia):
    if ((df_dia['isPivot']==2) & ((( df_dia['SMA100']-df_dia['Low']<=2 ) | ( df_dia['SMA200']-df_dia['Low']<=2 )) | (df_dia['SMA20']<df_dia['SMA40']) ) ):    
        return 1
    else:
        return 0
df_dia['zbarata'] = df_dia.apply(zonabarata, axis=1)

In [9]:
df_hora_final = pd.merge(df_hora, df_dia[['fecha','companyName', 'zbarata']], on = ['companyName','fecha'], how='left')

In [10]:
def pga(df_hora_final):
    if ((df_hora_final['zbarata']==1) & (df_hora_final['isPivot']==2)):
        return 1
    else:
        return 0
df_hora_final['pfuerte'] = df_hora_final.apply(pga, axis=1)

In [11]:
#vela verde
df_hora_final['Vela_Verde'] = df_hora_final['Close'] > df_hora_final['Open']

### Detect Price Channel

In [12]:
def collect_channel(candle, backevalTrend, trendH):

    localdf = dfpl.loc[candle-backevalTrend:candle] #tomar en cuenta el backcandles
    highs = localdf.loc[localdf.index.isin(trendH)].High.values
    idxhighs = localdf.loc[localdf.index.isin(trendH)].High.index
    
    if len(highs)>=2:
        sl_highs, interc_highs, r_value_h, _, _ = stats.linregress(idxhighs,highs)    
        return(sl_highs, interc_highs, r_value_h**2)
    else:
        return(0,0,0)

In [13]:
def backcandle(candle):
    
    trendH = []
    backeval = 0
    val = 0
    
    dfpl['PM40_PM20'] = dfpl['SMA40']-dfpl['SMA20']
    idinicial = dfpl.index[0]
    velas = candle - idinicial
    
    if(dfpl.loc[candle,"PM40_PM20"]>0):        
        idx_negativo = dfpl.loc[:candle, 'PM40_PM20'][::-1].lt(0).idxmax()
        #print("hito1")
        if (idx_negativo==candle):
            backeval = candle - velas
        else:
            backeval = idx_negativo - 4
    #print("hito2")
  
    val = candle - backeval
    
    #print("hito3")
    #print("idx_negativo:",idx_negativo)
    
    if(val>velas):
        backeval=candle-velas
    #print("hito4")

    #print("backeval:",backeval)
        
    if((dfpl.loc[backeval,"PM40_PM20"]>0) & (val<velas)):
        idx_negativo = dfpl.loc[:backeval, 'PM40_PM20'][::-1].lt(0).idxmax()
        backeval = idx_negativo - 4    
    #print("hito5")

    if val>velas:
        backeval=candle-velas
        
    #print("hito6")

    #ini = candle - backeval
    ini = backeval
    trendprev=dfpl.loc[ini:candle]
    trend = trendprev.reset_index(drop=False)

    idxhighs = trend.index.to_numpy().reshape(-1,1) # Convertir X a una matriz 2D
    highs = trend.High.values
    modelo1 = LinearRegression()
    modelo1.fit(idxhighs, highs)
    # Obtener la predicción de la línea de regresión
    Y_pred = modelo1.predict(idxhighs)
    # Calcular la distancia solo de los puntos que están POR ENCIMA de la línea
    trend['Distancia'] = trend["High"] - Y_pred  # Diferencia entre el valor real y la predicción
    # Filtrar los puntos que están por encima (donde Y > Y_pred, es decir, Distancia > 0)
    df_encima = trend[trend['Distancia'] > 0]
    puntos_mas_lejanos = df_encima.nlargest(2, 'Distancia')   #2 puntos mas lejanos    
    trendH = puntos_mas_lejanos["index"].tolist()
    
    puntos_mas_lejanos2 = trend.tail(10).nlargest(2, 'Distancia')
    trendH2 = puntos_mas_lejanos2["index"].tolist()
    
    trendH.extend(trendH2)
    trendH.append(candle)  #add ultimo candle
    #print(trendH)
    return trendH, backeval

### Detect Break Out

In [14]:
def trendChannel(row,candle, backeval, window, sl_highs, interc_highs):

    if (candle-backeval-window)<0:
        return np.nan
    ini=candle-backeval
    fin=candle+window
    trendcurrhigh = np.where(np.logical_or((row.index > fin),  (row.index <  ini)), np.nan, (sl_highs*row.index + interc_highs))
    return trendcurrhigh

In [15]:
def isBreakOut(candleEval,backcandles, window, start, candle):
    if (candleEval-backcandles-window)<0:
        return 0

    if candleEval==start:
        prev_idx = candleEval
    else:
        prev_idx = candleEval-1
        
    prev_high = dfpl.loc[prev_idx].High
    prev_low = dfpl.loc[prev_idx].Low
    prev_close = dfpl.loc[prev_idx].Close
    
    curr_idx = candleEval
    curr_high = dfpl.loc[candleEval].High
    curr_low = dfpl.loc[candleEval].Low
    curr_close = dfpl.loc[candleEval].Close
    curr_open = dfpl.loc[candleEval].Open

    trend_prevhigh=dfpl.loc[prev_idx].trendcurrhigh

    trend_currhigh=dfpl.loc[candleEval].trendcurrhigh
    
    if ( prev_low < trend_prevhigh and
        prev_close > trend_prevhigh and
        curr_open > trend_currhigh and
        curr_close > trend_currhigh and
        candleEval>candle): #and r_sq_h > 0.9
        return 2

    elif (curr_open > trend_currhigh and
        curr_close > trend_currhigh and candleEval>candle):
        return 3
    
    else:
        return np.nan

In [16]:
def breakpointpos(x):
    if x['isBreakOutIni'] in (2,3):
        return x['Low']+3e-3
    elif x['isBreakOutIni']==1:
        return x['High']-3e-3
    else:
        return np.nan

##### Hallando casos de Ruptura del Canal Bajista

In [17]:
def revisionVelas(dfpl, backeval,window, candle):
    #Revision de Velas
    #Promedio del volumen de 50 velas antes del posible caso
    promVol = dfpl.loc[i-backeval:i]["Volume"].mean()
    #puede darse cualquiera de estas condiciones: 
    # - que la vela sea verde, que tenga volumen, este por encima de la linea trend y que tenga cuerpo (cierra por encima de su tercera parte)
    # - que la vela sea envolvente, alcista y que este por encima de la linea trend
    j=candle+1
    k=candle+window
    ind_vela=0
    
    maxindex = dfpl.index[-1]
    if (k>maxindex):
        k=maxindex
    
    #Evaluacion de velas ALCISTAS
    dfpl["cdlengulfing"] = ta.CDLENGULFING(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlhammer"] = ta.CDLHAMMER(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlmorningstar"] = ta.CDLMORNINGSTAR(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlpiercing"] = ta.CDLPIERCING(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlclosingmarubozu"] = ta.CDLCLOSINGMARUBOZU(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlmarubozu"] = ta.CDLMARUBOZU(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdl3whitesoldiers"] = ta.CDL3WHITESOLDIERS(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlharami"] = ta.CDLHARAMI(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlharamicross"] = ta.CDLHARAMICROSS(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlinvertdhammer"] = ta.CDLINVERTEDHAMMER(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlladderbottom"] = ta.CDLLADDERBOTTOM(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])

    dfpl['Vela_Verde'] = dfpl['Close'] > dfpl['Open']
    # Calcular el nivel de la tercera parte
    dfpl['Third_Level'] = dfpl['Open'] + (dfpl['High'] - dfpl['Low']) / 3
    # Verificar si la vela cierra por encima de su tercera parte, vela con cuerpo
    dfpl['Above_Third'] = dfpl['Close'] > dfpl['Third_Level']

    while j<=k:            
        
        vol = dfpl.loc[j,"Volume"]
        above_Third = dfpl.loc[j,"Above_Third"]
        vela_Verde = dfpl.loc[j,"Vela_Verde"]
        lineTrend = dfpl.loc[j,'trendcurrhigh']
        high = dfpl.loc[j,'High']
        close = dfpl.loc[j,'Close']

        cdlengulfing = dfpl.loc[j,"cdlengulfing"]
        cdlhammer = dfpl.loc[j,"cdlhammer"]
        cdlmorningstar = dfpl.loc[j,"cdlmorningstar"]
        cdlpiercing = dfpl.loc[j,"cdlpiercing"]
        cdlclosingmarubozu = dfpl.loc[j,"cdlclosingmarubozu"]
        cdlmarubozu = dfpl.loc[j,"cdlmarubozu"]
        cdl3whitesoldiers = dfpl.loc[j,"cdl3whitesoldiers"]
        cdlharami = dfpl.loc[j,"cdlharami"]
        cdlharamicross = dfpl.loc[j,"cdlharamicross"]
        cdlinvertdhammer = dfpl.loc[j,"cdlinvertdhammer"]
        cdlladderbottom = dfpl.loc[j,"cdlladderbottom"]

        #vela consecutiva alcista
        candle_up = (
                     dfpl.loc[j,"High"]> dfpl.loc[j-1,"High"]
                     and dfpl.loc[j-1,"High"]> dfpl.loc[j,"Low"]
                     and dfpl.loc[j,"Low"]> dfpl.loc[j-2,"High"]
                     and dfpl.loc[j-2,"High"]> dfpl.loc[j-1,"Low"]
                     and dfpl.loc[j-1,"Low"]> dfpl.loc[j-3,"High"]
                     and dfpl.loc[j-3,"High"]> dfpl.loc[j-2,"Low"]
                     and dfpl.loc[j-2,"Low"]> dfpl.loc[j-3,"Low"]
                    )

        if (candle_up==True and close>lineTrend):
            ind_vela=ind_vela+1
            
        j=j+1
    
    return 1 #1 para mostrar todo ind_vela para que funcione el filtro

In [18]:
df_casos = pd.DataFrame()
caso = 0
ticker2 = ""
window=5
backeval=75
backevalTrend=0
dfpl = pd.DataFrame
trendH = []
trendL = []
cant = 0
for i, row in df_hora_final.iterrows():
    #if (i==83382):
    if(df_hora_final['pfuerte'][i]==1): #posibles casos 
        print("posible caso:",i, df_hora_final['companyName'][i])
        cant = 0
        candle = i
        cnt = 0
        valiniHigh = df_hora_final.loc[candle,"High"]
        #valiniLow = df.loc[candle,"Low"]
        ticker = df_hora_final['companyName'][i]
        #Reinicio de casos por company
        if ticker2 != ticker:
            ticker2 = ticker
            caso = 0
        dfpl = df_hora_final[(df_hora_final.companyName==ticker)].loc[i-backeval:i+backeval].copy()

        #print("rowMin:", dfpl.index[0],", rowMax:", dfpl.index[-1])
                    
        h=1
        l=1

        cant = cant +1
        cantHorasTrend=0
        #print("opcionH:",h,"opcionL:",l)
        trendH, backevalTrend = backcandle(candle)
        #print ("-->trendH:",trendH)
        #print ("-->backevalTrend:",backevalTrend)    
        
        #cantHorasTrend = (dfpl.loc[candle,"datetime"] - dfpl.loc[backevalTrend,"datetime"]).total_seconds() / 3600

        #if (cantHorasTrend>48): #Verificacion de caida por lo menos 2 dias
        backevalTrend2 = candle - backevalTrend
    
                    
        #revision de trend negativo indica que esta cayendo, solo me quedo con estos casos
        sl_highs, interc_highs, r_sq_h = collect_channel(candle, backevalTrend2, trendH)
                

        #print("======:", candle)
        #print("===trendH===")
        #print(trendH)
        #print("===trendL===")
        #print(trendL)
        dfpl.loc[trendH, "trendH"] = 1
    
        
        #print("sl_highs:", sl_highs, ", sl_lows:", sl_lows, ", r_sq_h:", r_sq_h)
        if (sl_highs<=0): #solo tendencias a la baja and ind_vela>0 sl_lows<0 and r_sq_h>=0.7  and sl_highs<=-0.1 and ind_vela>0
            #print("paso1")
            #crear linea de tendencia high y low
            dfpl['trendcurrhigh']  = trendChannel(dfpl,candle,backevalTrend2,window,sl_highs,interc_highs)
            
            #ind_vela = revisionVelas(dfpl, backeval,window, candle)
            ind_vela = 1
            promVol = dfpl.loc[i-backeval:i]["Volume"].mean()

            #print("ind_vela:",ind_vela)
            if (ind_vela>0):
                #print("paso2")
                dfpl["bearishSlope"] = np.nan #pendiente bajista en vacio
                dfpl.loc[i, "bearishSlope"] = 1 #pendiente bajista solo al punto evaluado            
                dfpl["ind_posicion"] = 0
                dfpl.loc[(dfpl.index < i), 'ind_posicion'] = -1
                dfpl.loc[(dfpl.index > i), 'ind_posicion'] = 1
                dfpl["promVol"] = promVol
                #dfpl["sl_highs"] = sl_highs
                #dfpl["r_sq_h"] = r_sq_h
        
                start = dfpl.index[0]
                dfpl["isBreakOut"] = [isBreakOut(candleEval, backevalTrend2, window, start, candle) for candleEval in dfpl.index]
                # Solo me quedo con el primer BREAK OUT
                cnt = dfpl.query("isBreakOut in (2,3) and Close>Open").shape[0] 
                id=0
                id2=0
                #print("paso3")
                if cnt>0:
                    #print("paso4")
                    id = dfpl.query("isBreakOut in (2,3) and Close>Open").index[0]                        
                    #dfpl['isBreakOutIni'] = np.where(id!=dfpl.index, np.nan, dfpl['isBreakOut'])                        
                    dfpl.loc[id,'isBreakOutIni'] = 1
                    #print("paso5")
                    k=0
                    while (k<=3):
                        dfpl['breakpointpos'] = dfpl.apply(lambda row: breakpointpos(row), axis=1)
                        cnt2 = dfpl.query("index>@id and isPivot==1").shape[0]
                        #print("paso6")
                        if cnt2>0:
                            id2 = dfpl.query("index>@id and isPivot==1").index[0]
                            dfpl.loc[id2,'isBreakOutFinal'] = 1
                            k=4
                            #print("paso7")
                        else:
                            #revisar 25 velas mas
                            #print("paso8")
                            idfinal = dfpl.index[-1] 
                            idfinal2 = idfinal+25
                            if idfinal2 in df_hora_final[df_hora_final['companyName']==ticker].index:
                                dfpl2 = df_hora_final[(df_hora_final.companyName==ticker)].loc[idfinal+1:idfinal2].copy()
                                dfpl = pd.concat([dfpl, dfpl2],ignore_index=False)
                                #cnt2 = dfpl.query("index>@id and isPivot==1").shape[0]                    
                                #if cnt2>0:
                                #    id2 = dfpl.query("index>@id and isPivot==1").index[0]
                                #    dfpl.loc[id2,'isBreakOutFinal'] = 1
                                #else:
                                #    dfpl['isBreakOutFinal'] = np.nan
                            else:
                                dfpl['isBreakOutFinal'] = np.nan  
                            k=k+1
                    
                else:
                    dfpl['isBreakOutIni'] = np.nan                        
                    dfpl['breakpointpos'] = np.nan
        
                if cnt>0:
                    #INSERT CASO
                    caso=caso+1   
                    print("insert caso:",i, df_hora_final['companyName'][i])
                    dfpl["caso"] = caso                        
                    if len(df_casos)<1:
                        df_casos = dfpl
                    else:
                        df_casos = pd.concat([df_casos, dfpl],ignore_index=False)

posible caso: 665 SPY


insert caso: 665 SPY
posible caso: 1495 SPY
insert caso: 1495 SPY
posible caso: 1997 SPY


posible caso: 2160 SPY
insert caso: 2160 SPY
posible caso: 2437 SPY


insert caso: 2437 SPY
posible caso: 3857 META
posible caso: 4196 META


insert caso: 4196 META
posible caso: 5084 META
insert caso: 5084 META
posible caso: 5487 META


...

insert caso: 109459 APP
posible caso: 109592 APP
insert caso: 109592 APP
posible caso: 110261 UBER


insert caso: 110261 UBER
posible caso: 110612 UBER


insert caso: 110612 UBER
posible caso: 110946 UBER
insert caso: 110946 UBER
posible caso: 111629 UBER


posible caso: 111952 UBER
insert caso: 111952 UBER
posible caso: 112598 UBER


insert caso: 112598 UBER


In [19]:
path = 'data/pfuerte.txt'

# check whether the file exists
if os.path.exists(path):
    os.remove(path)
else:
    # if the file does not exist.
    print("File does not exists. File needs to be created.")

#export DataFrame to text file
with open(path, 'a') as f:
    df_casos.to_csv(path, header=True, index=None, sep='\t', mode='w')


##### BACKTESTING
###### Usando paquete backtesting.py

In [20]:
df_casos2 = df_casos.query("isBreakOutIni==1 or isBreakOutFinal==1").copy()

In [21]:
# =============== BACKTESTING ================= #
# obteniendo dataframe backtesting
df_backTesting = pd.merge(df_hora_final, df_casos2[['datetime','companyName', 'isBreakOutIni','isBreakOutFinal','caso']], on = ['companyName','datetime'], how='left')
# datos de trade se asigna una vela anterior debido a que backtesting.py toma entra como la siguiente vela
df_backTesting['isBreakOutIni'] = df_backTesting['isBreakOutIni'].shift(-1)
df_backTesting['isBreakOutFinal'] = df_backTesting['isBreakOutFinal'].shift(-1)
#modificamos indice con el compo datetime
df_backTesting.set_index('datetime', inplace=True)

In [22]:
#Strategia
class strategyPisoFuerte(Strategy):    
    def init(self):        
        self.breakout_entry = self.I(lambda x: x, self.data.isBreakOutIni) #Indica la señal entrada al backtesting
        self.breakout_exit = self.I(lambda x: x, self.data.isBreakOutFinal) #Indica la señal salida al backtesting

    def next(self):       
        if self.breakout_entry[-1] == 1:
            self.buy()  #COMPRA
        elif self.breakout_exit[-1] == 1 and self.position:
            self.position.close() #CIERRE

In [23]:
# Por cada TIcket
my_stats = {}
estadisticas = pd.DataFrame()          # lista para ticker
tradesprev = pd.DataFrame()     # lista para cada caso por ticker
for ticker in df_backTesting['companyName'].unique():
    ticker_data = df_backTesting[df_backTesting['companyName'] == ticker].copy() #Filtro por cada ticker
    bt = Backtest(ticker_data, strategyPisoFuerte, cash=10_000)
    my_stats = bt.run()
    listTrades = my_stats['_trades']
    tradesdf = pd.DataFrame(listTrades) #trades
    tradesdf['Ticker'] = ticker
    stats_filter={
        'Ticker':ticker,
        'EntryTime':my_stats['Start'],
        'ExitTime':my_stats['End'],
        'Return [%]': my_stats['Return [%]'],
        'CAGR [%]': my_stats['Return (Ann.) [%]'],
        'Sharpe Ratio': my_stats['Sharpe Ratio'],
        'Max. Drawdown [%]': my_stats['Max. Drawdown [%]'],
        'Win Rate [%]': my_stats['Win Rate [%]'],
        '# Trades': my_stats['# Trades'],
        'Expectancy [%]': my_stats['Expectancy [%]'],
        'Profit Factor': my_stats['Profit Factor'],
        'Duration': my_stats['Duration'],
        'Avg. Trade [%]':my_stats['Avg. Trade [%]'],
        'Max. Trade Duration':my_stats['Max. Trade Duration'],
        'Avg. Trade Duration':my_stats['Avg. Trade Duration']
    }
    stat=pd.DataFrame([stats_filter])
    if tradesprev.shape[0]==0:
        tradesprev = tradesdf.copy()
        estadisticas=stat.copy()
    else:
        tradesprev = pd.concat([tradesprev, tradesdf],ignore_index=True)
        estadisticas=pd.concat([estadisticas, stat],ignore_index=True)
    # Mostramos
    #bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")

Backtest.run:   0%|          | 0/2004 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/2784 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/2744 [00:00<?, ?bar/s]

/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=2448: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
                                                       

Backtest.run:   0%|          | 0/2820 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/3361 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3391 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/1437 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3211 [00:00<?, ?bar/s]

/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=993: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


In [24]:
ini = df_casos2[(df_casos2['isBreakOutIni']==1)].copy()
df_casos3_ini=ini.rename(columns={'Open': 'EntryPrice', 'datetime': 'EntryTime', 'companyName': 'Ticker'})[['EntryPrice','Ticker','caso','EntryTime']]
final = df_casos2[(df_casos2['isBreakOutFinal']==1)].copy()
df_casos3_final=final.rename(columns={'Open': 'ExitPrice', 'datetime': 'ExitTime', 'companyName': 'Ticker'})[['ExitPrice','Ticker','caso','ExitTime']]

In [25]:
df_casos3=df_casos3_ini.merge(df_casos3_final,on=['Ticker','caso'],how='outer')

In [26]:
trades = pd.merge(df_casos3, tradesprev, on = ['EntryTime','Ticker'], how='left')

In [27]:
def exitPrice(x):
    if pd.isna(x['ExitPrice_x']):
        return x['ExitPrice_y']
    else:
        return x['ExitPrice_x']
trades['ExitPrice'] = trades.apply(lambda row: exitPrice(row), axis=1)

In [28]:
trades = trades.rename(columns={'EntryPrice_x': 'EntryPrice'})
trades = trades.drop(['EntryPrice_y','ExitPrice_x', 'ExitPrice_y'], axis=1)

In [29]:
def exitTime(x):
    if pd.isna(x['ExitTime_x']):
        return x['ExitTime_y']
    else:
        return x['ExitTime_x']
trades['ExitTime'] = trades.apply(lambda row: exitTime(row), axis=1)

In [30]:
def duration(x):
    if pd.isna(x['Duration']):
        if pd.isna(x['ExitTime']):
            return pd.NaT
        else:
            return x['ExitTime']-x['EntryTime']
    else:
        return x['Duration']
trades['Duration'] = trades.apply(lambda row: duration(row), axis=1)

In [31]:
trades = trades.drop(['ExitTime_x', 'ExitTime_y'], axis=1)

In [32]:
trades=trades[['Ticker','EntryTime','ExitTime','EntryPrice','ExitPrice','Duration','Size','EntryBar','ExitBar','ReturnPct','caso']]

In [33]:
for item in estadisticas['Ticker'].unique():
    count = trades[trades["Ticker"] == item].shape[0]
    estadisticas.loc[estadisticas['Ticker'] == item, '# Trades'] = count

In [34]:
path = 'data/backtesting'

os.makedirs(path, exist_ok=True)

path_trades = os.path.join(path, 'trades_pfuerte.txt')
path_estadisticas = os.path.join(path, 'estadisticas_pfuerte.txt')

for file_path in [path_trades, path_estadisticas]:
    if os.path.exists(file_path):
        os.remove(file_path)
    else:
        print(f"File does not exist: {file_path}. It will be created.")

trades.to_csv(path_trades, header=True, index=False, sep='\t', mode='w')
estadisticas.to_csv(path_estadisticas, header=True, index=False, sep='\t', mode='w')